In [21]:
import torch

In [22]:
if torch.cuda.is_available():
    print("cuda!")

cuda!


In [23]:
!pip install sentence_transformers

In [69]:
import numpy as np
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util
from google.colab import drive
import csv

# Mount Google Drive
drive.mount('/content/drive')

# Define the path where you want to save the embeddings
save_path = '/content/drive/My Drive/embeddings_OOD.npy'

# Check if GPU is available and set the device accordingly
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load your dataset
#IMDB_data = pd.read_csv("/content/IMDB_Dataset.csv", sep=',')
# IMDB_data = pd.read_csv("/content/IMDB Dataset.csv", sep=',', encoding='utf-8')
# print(IMDB_data.head)
# # Select a subset of the data
# corpus = IMDB_data['review'].tolist()
# # Load the model and move it to the device (GPU if available)
# model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1", device=device)

# # Encode the corpus with the model on the specified device
# embeddings = model.encode(corpus, device=device)
# print(embeddings.shape)

# # Save embeddings to the specified path
# np.save(save_path, embeddings)
# print(f'Embeddings saved to {save_path}')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
from sklearn.datasets import fetch_20newsgroups

data_train = fetch_20newsgroups(
    subset='train',
    shuffle=True,
    random_state=42
)
IMDB_data = data_train.data

In [51]:
IMDB_data = pd.DataFrame(IMDB_data)

In [58]:
IMDB_data[0]

0        From: lerxst@wam.umd.edu (where's my thing)\nS...
1        From: guykuo@carson.u.washington.edu (Guy Kuo)...
2        From: twillis@ec.ecn.purdue.edu (Thomas E Will...
3        From: jgreen@amber (Joe Green)\nSubject: Re: W...
4        From: jcm@head-cfa.harvard.edu (Jonathan McDow...
                               ...                        
11309    From: jim.zisfein@factory.com (Jim Zisfein) \n...
11310    From: ebodin@pearl.tufts.edu\nSubject: Screen ...
11311    From: westes@netcom.com (Will Estes)\nSubject:...
11312    From: steve@hcrlgw (Steven Collins)\nSubject: ...
11313    From: gunning@cco.caltech.edu (Kevin J. Gunnin...
Name: 0, Length: 11314, dtype: object

In [60]:
IMDB_data[0]= IMDB_data[0].str.lower()

In [28]:
import re
def remove_html_tags(text):
    pattern = re.compile("<.*?>")
    return pattern.sub(r"", text)

In [61]:
IMDB_data[0] = IMDB_data[0].apply(remove_html_tags)

In [30]:
def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

In [62]:
IMDB_data[0] = IMDB_data[0].apply(remove_url)

In [32]:
import string
exclude = string.punctuation
def remove_punctuation(text):
    for char in exclude:
        text = text.replace(char, "")
    return text

In [63]:
IMDB_data[0] = IMDB_data[0].apply(remove_punctuation)

In [34]:
chat_word = {
    'AFAIK': 'As Far As I Know',
    'AFK': 'Away From Keyboard',
    'ASAP': 'As Soon As Possible',
    'ATK': 'At The Keyboard',
    'ATM': 'At The Moment',
    'A3': 'Anytime, Anywhere, Anyplace',
    'BAK': 'Back At Keyboard',
    'BBL': 'Be Back Later',
    'BBS': 'Be Back Soon',
    'BFN': 'Bye For Now',
    'B4N': 'Bye For Now',
    'BRB': 'Be Right Back',
    'BRT': 'Be Right There',
    'BTW': 'By The Way',
    'B4': 'Before',
    'CU': 'See You',
    'CUL8R': 'See You Later',
    'CYA': 'See You',
    'FAQ': 'Frequently Asked Questions',
    'FC': 'Fingers Crossed',
    'FWIW': "For What It's Worth",
    'FYI': 'For Your Information',
    'GAL': 'Get A Life',
    'GG': 'Good Game',
    'GN': 'Good Night',
    'GMTA': 'Great Minds Think Alike',
    'GR8': 'Great!',
    'G9': 'Genius',
    'IC': 'I See',
    'ICQ': 'I Seek you (also a chat program)',
    'ILU': 'ILU: I Love You',
    'IMHO': 'In My Honest/Humble Opinion',
    'IMO': 'In My Opinion',
    'IOW': 'In Other Words',
    'IRL': 'In Real Life',
    'KISS': 'Keep It Simple, Stupid',
    'LDR': 'Long Distance Relationship',
    'LMAO': 'Laugh My A.. Off',
    'LOL': 'Laughing Out Loud',
    'LTNS': 'Long Time No See',
    'L8R': 'Later',
    'MTE': 'My Thoughts Exactly',
    'M8': 'Mate',
    'NRN': 'No Reply Necessary',
    'OIC': 'Oh I See',
    'PITA': 'Pain In The A..',
    'PRT': 'Party',
    'PRW': 'Parents Are Watching',
    'QPSA?': 'Que Pasa?',
    'ROFL': 'Rolling On The Floor Laughing',
    'ROFLOL': 'Rolling On The Floor Laughing Out Loud',
    'ROTFLMAO': 'Rolling On The Floor Laughing My A.. Off',
    'SK8': 'Skate',
    'STATS': 'Your sex and age',
    'ASL': 'Age, Sex, Location',
    'THX': 'Thank You',
    'TTFN': 'Ta-Ta For Now!',
    'TTYL': 'Talk To You Later',
    'U': 'You',
    'U2': 'You Too',
    'U4E': 'Yours For Ever',
    'WB': 'Welcome Back',
    'WTF': 'What The F...',
    'WTG': 'Way To Go!',
    'WUF': 'Where Are You From?',
    'W8': 'Wait...',
    '7K': 'Sick:-D Laugher',
    'TFW': 'That feeling when',
    'MFW': 'My face when',
    'MRW': 'My reaction when',
    'IFYP': 'I feel your pain',
    'TNTL': 'Trying not to laugh',
    'JK': 'Just kidding',
    'IDC': "I don't care",
    'ILY': 'I love you',
    'IMU': 'I miss you',
    'ADIH': 'Another day in hell',
    'ZZZ': 'Sleeping, bored, tired',
    'WYWH': 'Wish you were here',
    'TIME': 'Tears in my eyes',
    'BAE': 'Before anyone else',
    'FIMH': 'Forever in my heart',
    'BSAAW': 'Big smile and a wink',
    'BWL': 'Bursting with laughter',
    'BFF': 'Best friends forever',
    'CSL': "Can't stop laughing"
}

In [35]:
def short_conv(text):
    new_text = []  # Initialize an empty list to hold the processed words
    for w in text.split():  # Split the input text into words and iterate over them
        if w.upper() in chat_word:  # Check if the uppercase version of the word is in the chat_word dictionary
            new_text.append(chat_word[w.upper()])  # If it is, append the full form from the dictionary to new_text
        else:
            new_text.append(w)  # If it is not, append the original word to new_text
    return " ".join(new_text)  # Join the processed words into a single string and return it

In [64]:
IMDB_data[0] = IMDB_data[0].apply(short_conv)

In [38]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [39]:
from nltk.corpus import stopwords
stop_words = stopwords.words("english")


In [40]:
def remove_stopwords(text):
    new_text=[]
    for word in text.split():
        if word in stop_words:
            new_text.append('')
        else:
            new_text.append(word)

    x = new_text[:]  # Create a copy of new_text
    new_text.clear()  # Clear the original new_text list
    return " ".join(x)  # Join the copied list x into a single string separated by spaces and return it

In [65]:
IMDB_data[0] = IMDB_data[0].apply(remove_stopwords)

In [42]:
# Removing Emojis
def remove_emoji(text):
    emoji_pattern=re.compile("["
                             u"\U0001F600-\U0001F64F" #emoticons
                             u"\U0001F300-\U0001F5FF" #symbols, pictograph
                              u"\U0001F680-\U0001F6FF" #transport and map symbol
                              u"\U0001F1E0-\U0001F1FF" #flags(IOS)
                              u"\U00002702-\U000027B0"
                              u"\U00002FC2-\U0001F251"
                             "]+",flags=re.UNICODE)
    return emoji_pattern.sub(r'',text)

In [66]:
IMDB_data[0] = IMDB_data[0].apply(remove_emoji)

In [68]:
IMDB_data.head()

,0
0,lerxstwamumdedu wheres thing subject car ...
1,guykuocarsonuwashingtonedu guy kuo subject si...
2,twillisececnpurdueedu thomas e willis subject...
3,jgreenamber joe green subject weitek p9000 o...
4,jcmheadcfaharvardedu jonathan mcdowell subjec...


In [70]:
# Select a subset of the data
corpus = IMDB_data[0].tolist()
# Load the model and move it to the device (GPU if available)
model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1", device=device)

# Encode the corpus with the model on the specified device
embeddings = model.encode(corpus, device=device)
print(embeddings.shape)

# Save embeddings to the specified path
np.save(save_path, embeddings)
print(f'Embeddings saved to {save_path}')

(11314, 384)
Embeddings saved to /content/drive/My Drive/embeddings_OOD.npy
